In [ ]:
import pandas as pd

# Load the datasets
final_movies = pd.read_csv('Data/final_dataset.csv')
new_ratings = pd.read_csv('Data/ratings.csv')
genome = pd.read_csv('Data/new_genome.csv')

# Merge the genome dataset with the final_movies dataset on movieId
genome_merged = pd.merge(genome, final_movies[['movieId', 'title', 'genres']], on='movieId')

# Prepare the collaborative filtering data
collab_data = new_ratings[['userId', 'movieId', 'rating']]

# Prepare the content-based filtering data
content_data = genome_merged[['movieId', 'title', 'genres', 'tagId', 'relevance']]

# Ensure no missing values
collab_data = collab_data.dropna()
content_data = content_data.dropna()


: 